# Untitled

## How to run DDD with CSDID in Stata

Here I will show a small example, using the mpdta datafile.

First the setup

In [1]:
qui:ssc install frause, replace
frause mpdta, clear
** Assume two groups, based on lpop
gen g1=lpop>3.3
** Now use csdid to estimate the model, without covariates
csdid lemp if g1==0, ivar(countyreal) time(year) gvar(first_treat) saverif(f1) replace
csdid lemp if g1==1, ivar(countyreal) time(year) gvar(first_treat) saverif(f2) replace

(Written by R.              )

File f1.dta will be replaced

.

.

.

...

.

.

.

.

.
file f2.dta saved

Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 1,205
Outcome model  : regression adjustment
Treatment model: none
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0036454    .017134    -0.21   0.832    -.0372275    .0299367
 t_2003_2005 |   -.016655   .0232565    -0.72   0.474    -.0622369    .0289269
 t_2003_2006 |  -.0705129   .0266075    -2.65   0.008    -.1226626   -.0183631
 t_2003_2007 |  -.0725062   .0217586    -3.33   0.001    -.1151523   -.0298601
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |   -.000845   .0293484    -0.03   0.977    -.0583669    .0566768
 t_2004_2005 |   .0069579   .0178941     0.39

Now there are two files with the corresponding RIFs, so I can get
aggregations for each separately

In [2]:
use f1, clear
csdid_stats simple, save
ren ATT ATT_g1
save f1, replace

use f2, clear
csdid_stats simple, save
ren ATT ATT_g2
save f2, replace

(Written by R.              )

------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         ATT |  -.0526544   .0223489    -2.36   0.018    -.0964575   -.0088513
------------------------------------------------------------------------------
file f1.dta saved
(Written by R.              )


------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         ATT |     -.0328   .0094731    -3.46   0.001     -.051367    -.014233
------------------------------------------------------------------------------
file f2.dta saved

Next I need to merge them together

In [3]:
use f1, clear
merge 1:1 countyreal using f2
sum ATT_g1 ATT_g2

(Written by R.              )

    Result                      Number of obs
    -----------------------------------------
    Not matched                           500
        from master                       259  (_merge==1)
        from using                        241  (_merge==2)

    Matched                                 0  (_merge==3)
    -----------------------------------------

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
      ATT_g1 |        259   -.0526544    .3603682  -3.026657    2.10358
      ATT_g2 |        241      -.0328    .1473684  -.5103483   .7988653

Now you have both files together, with different treatment effects. To
create a table you can use the command csdid_rif, to compare both
results, and make test for differences

In [4]:
csdid_rif ATT_g1 ATT_g2
test ATT_g1= ATT_g2

------------------------------------------------------------------------------
             |               Robust
             | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
      ATT_g1 |  -.0526544   .0223489    -2.36   0.018    -.0964575   -.0088513
      ATT_g2 |     -.0328   .0094731    -3.46   0.001     -.051367    -.014233
------------------------------------------------------------------------------

 ( 1)  ATT_g1 - ATT_g2 = 0

           chi2(  1) =    0.67
         Prob > chi2 =    0.4134